# Cross 1L TL

In [16]:
import csv
import os
import shutil
import subprocess

import pyautogui
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.monitor import Monitor
from sumo_rl import SumoEnvironment

from evaluate import evaluate
from helper_functions import linear_schedule
from reward_functions import tyre_pm_reward

In [17]:
TEST_NUM = 1

## Create Environment and Instantiate Agent

In [18]:
csv_dir = os.path.join("outputs","cross1ltl",f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)
    
env_params = {
    "net_file": os.path.join("nets","simple_nets","cross1ltl","net.net.xml"),
    "route_file": os.path.join("nets","simple_nets","cross1ltl","input_routes.rou.xml"),
    "out_csv_name": os.path.join(csv_dir,"ppo"),
    "num_seconds": 1200,
    "single_agent": True,
    "reward_fn": tyre_pm_reward,
    "sumo_seed": 42,
}
env = SumoEnvironment(**env_params)
check_env(env)
env = Monitor(env)  # wrap env to know episode reward, length, time

In [19]:
# Using hyperparams from https://arxiv.org/abs/1707.06347 Table 5, except for `n_steps`,
# which is from https://github.com/LucasAlegre/sumo-rl/blob/master/experiments/sb3_grid4x4.py

model = PPO(
    "MlpPolicy",
    env,
    learning_rate=linear_schedule(2.5e-4),
    n_steps=256,
    batch_size=256,
    n_epochs=3,
    clip_range=linear_schedule(0.1),
    ent_coef=0.01,
    vf_coef=1,
    max_grad_norm=0.9,
    tensorboard_log=os.path.join("logs","cross1ltl"),
    verbose=1
)

Using cuda device
Wrapping the env in a DummyVecEnv.


In [20]:
# Evaluate untrained random agent
csv_path = os.path.join(csv_dir, "untrained.csv")
tb_log_dir = os.path.join("logs","cross1ltl","eval_untrained")

mean_reward_untrained, std_reward_untrained = evaluate(model, env, csv_path, tb_log_dir, n_eval_episodes=1)

print(mean_reward_untrained)
print(std_reward_untrained)

-5401.889468
0.0


## Train and Evaluate Agent

In [ ]:
# Train agent
model.learn(total_timesteps=1e5)

In [ ]:
# Evaluate trained agent
csv_path = os.path.join(csv_dir, "trained.csv")
tb_log_dir = os.path.join("logs","2way-single-intersection",f"PPO_{TEST_NUM}","eval_trained")

mean_reward_trained, std_reward_trained = evaluate(model, env, csv_path, tb_log_dir, n_eval_episodes=1)

print(mean_reward_trained)
print(std_reward_trained)

In [ ]:
original = os.path.join("logs","2way-single-intersection","eval_untrained")
target = os.path.join("logs","2way-single-intersection",f"PPO_{TEST_NUM}","eval_untrained")

shutil.move(original, target)

### Record Results

In [ ]:
data = [env_params["net_file"], TEST_NUM, env.reward_fn.__name__, mean_reward_untrained, mean_reward_trained]

with open("test_results.csv", "a", encoding="UTF8", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(data)

### Save Model

In [ ]:
model.save(os.path.join("saved_models",f"PPO_2way_single_intersection_{TEST_NUM}"))

## Render Video

In [ ]:
env_params.update({"use_gui": True})
render_env = SumoEnvironment(**env_params)

check_env(render_env)
render_env.close()

render_env = Monitor(render_env)  # wrap env to know episode reward, length, time

In [ ]:
model = PPO.load(os.path.join("saved_models",f"PPO_2way_single_intersection_{TEST_NUM}"))

obs, info = render_env.reset()

folder_path = os.path.join("renders","2way-single-intersection",f"test_{TEST_NUM}")
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

VID_LENGTH = 750
for i in range(VID_LENGTH):
    actions, _ = model.predict(obs)
    obs, reward, terminated, truncated, info = render_env.step(actions)
    env.render()

    im = pyautogui.screenshot(
        os.path.join(folder_path,f"img{i}.jpg"),
        region=(0, 0, 2560, 1542)
    )

render_env.close()  # clean up

In [ ]:
subprocess.run([
    "ffmpeg", "-y", "-r", "5", "-i",
    f"renders\\2way-single-intersection\\test_{TEST_NUM}\\img%d.jpg",
    f"videos\\2way_single_intersection_{TEST_NUM}.mp4"
])

## Clean-up

In [ ]:
env.close()
render_env.close()